# Coleta de dados da busca em profundidade (DFS)

## Função DFS

In [ ]:
import os
import numpy as np
import csv
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import random
import time
import tracemalloc
from collections import deque
import pandas as pd
import sys

def dfs_resolver_labirinto_com_visualizacao(labirinto, inicio, fim):
    altura, largura = len(labirinto), len(labirinto[0])
    visitado = [[False for _ in range(largura)] for _ in range(altura)]
    pai = [[None for _ in range(largura)] for _ in range(altura)]

    pilha = deque([])
    nos_visitados_dfs = set()

    pilha.append(inicio)

    tracemalloc.start()
    tempo_inicio = time.perf_counter()

    caminho_encontrado = False
    while pilha:
        x, y = pilha.pop()

        if visitado[y][x]:
            continue

        visitado[y][x] = True
        nos_visitados_dfs.add((x,y))

        if (x, y) == fim:
            caminho_encontrado = True
            break

        for dx, dy in [(0,1), (0,-1), (1,0), (-1,0)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < largura and 0 <= ny < altura:
                if labirinto[ny][nx] == 1 and not visitado[ny][nx]:
                    pilha.append((nx, ny))
                    pai[ny][nx] = (x, y)

    tempo_fim = time.perf_counter()
    memoria_usada = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()

    caminho_final = []
    if caminho_encontrado:
        curr_x, curr_y = fim
        while (curr_x, curr_y) != inicio and pai[curr_y][curr_x] is not None:
            caminho_final.append((curr_x, curr_y))
            curr_x, curr_y = pai[curr_y][curr_x]
        caminho_final.append(inicio)
        caminho_final.reverse()

    return {
        "caminho": caminho_final,
        "nos_visitados": list(nos_visitados_dfs),
        "tempo_ms": (tempo_fim - tempo_inicio) * 1000,
        "memoria_bytes": memoria_usada,
        "comprimento": len(caminho_final) if caminho_encontrado else 0,
        "qtd_nos_visitados": len(nos_visitados_dfs)
    }

## Resolução de lote de labirintos

In [ ]:
def executar_dfs_em_lote(caminho_labirintos, tamanho_nome, output_csv):
    resultados = []

    for i in range(1, 21):
        lab_path = os.path.join(caminho_labirintos, f"labirinto_{i:02d}.npy")
        if not os.path.exists(lab_path):
            print(f"Labirinto não encontrado: {lab_path}")
            continue

        labirinto = np.load(lab_path)
        entrada = (0, 1)
        saida = (labirinto.shape[1] - 1, labirinto.shape[0] - 2)

        resultado = dfs_resolver_labirinto_com_visualizacao(labirinto, entrada, saida)
        print(f"Labirinto {i}/20")
        resultados.append([
            "DFS",
            tamanho_nome,
            i,
            round(resultado["tempo_ms"], 2),
            resultado["qtd_nos_visitados"],
            bool(resultado["comprimento"] > 0),
            resultado["comprimento"],
            round(resultado["memoria_bytes"] / 1024, 2)  # em KB
        ])

    # Salva CSV
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)
    with open(output_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            "algoritmo", "tamanho_labirinto", "indice_labirinto",
            "tempo_execucao_ms", "nos_visitados", "encontrou_caminho",
            "comprimento_caminho", "memoria_kb"
        ])
        writer.writerows(resultados)

    print(f"✅ Resultados salvos em: {output_csv}")


## Resolução do lote com labirintos pequenos

In [ ]:
executar_dfs_em_lote("labirintos/pequenos", "pequeno", "resultados/dfs/dfs_pequenos.csv")


## Resolução do lote com labirintos médios

In [ ]:
executar_dfs_em_lote("labirintos/medios", "medio", "resultados/dfs/dfs_medios.csv")


## Resolução do lote com labirintos médios-grandes

In [ ]:
executar_dfs_em_lote("labirintos/medio-grande", "medio-grande", "resultados/dfs/dfs_medios_grandes.csv")


## Resolução do lote com labirintos grandes

In [ ]:
executar_dfs_em_lote("labirintos/grandes", "grande", "resultados/dfs/dfs_grandes.csv")
